# NeMo Framework - Training a large language model

## Overview
Large language model (LLM) like ChatGPT possess astonishing versatility, being able to perform tasks such as induction, programming, translation, and more, with results comparable to or even superior to human experts. To learn how to pre-train a large language model (LLM). NVIDIA has introduced NeMo Framework that is capabilities to pre-process training data, distribute training across multiple GPUs efficiently.

Pre-trained language model is powerful in a variety of tasks but often lack the specialized focus needed for domain-specific applications. Therefore, to adapt the language model to a domain-specific task, fine-tuning can be employed. In this notebook, you will learn how to implement two type of tuning methods, **(1)Fine-tuning** and **(2)PEFT methods** like **LoRA** for adapting language model on specific downstream task using NVIDIA NeMo.

## Table of Contents

This course covers the below sections:
1. [Pre-training](#s1)
    - [1.1 Download dataset](#s1.1)
    - [1.2 Data preprocessing](#s1.2)
    - [1.3 Download pre-trained model for continued pre-training](#s1.3)
    - [1.4 Run pre-training](#s1.4)
    
    
2. [Instruction Tuning ](#s2)
    - [2.1 Download dataset: erhwenkuo/alpaca-data-gpt4-chinese-zhtw](#s2.1)
    - [2.2 Split the data into train, validation and test](#s2.2)
    - [2.3 Full parameter fine-tuning](#s2.3)
    - [2.4. Parameter Efficient Fine-tuning](#s2.4)


3. [Evaluation](#s3)

## 1. Pre-training <a name='s1'></a>

The initial phase of our process is concentrated on model pre-training, which serves as the primary stage for the model to acquire knowledge.

### 1.1 Download dataset <a name='s1.1'></a>

In [1]:
from datasets import load_dataset
dataset = load_dataset('erhwenkuo/wikinews-zhtw')['train']
dataset.to_json('./data/custom_dataset/json/wikinews-zhtw.jsonl', force_ascii=False)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Creating json from Arrow format: 100%|██████████| 10/10 [00:00<00:00, 67.04ba/s]


13914259

### 1.2 Data preprocessing <a name='s1.2'></a>

In [2]:
%%bash

export HF_TOKEN=???

mkdir -p data/custom_dataset/preprocessed

python /opt/NeMo/scripts/nlp_language_modeling/preprocess_data_for_megatron.py \
--input=data/custom_dataset/json/wikinews-zhtw.jsonl \
--json-keys=text \
--dataset-impl mmap \
--tokenizer-library=huggingface \
--tokenizer-type meta-llama/Llama-3.1-8B-Instruct \
--output-prefix=data/custom_dataset/preprocessed/wikinews \
--append-eod

[NeMo W 2024-12-06 06:58:50 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:280: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, allreduce_dgrad):
    
[NeMo W 2024-12-06 06:58:50 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:290: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-06 06:58:50 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:381: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-06 06:58:50 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:420: FutureWarning: `t

[NeMo I 2024-12-06 06:58:55 tokenizer_utils:184] Getting HuggingFace AutoTokenizer with pretrained_model_name: meta-llama/Llama-3.1-8B-Instruct
Vocab size: 128256
Output prefix: data/custom_dataset/preprocessed/wikinews
Time to startup: 1.6398766040802002


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[NeMo I 2024-12-06 06:58:57 tokenizer_utils:184] Getting HuggingFace AutoTokenizer with pretrained_model_name: meta-llama/Llama-3.1-8B-Instruct


Processed 100 documents (196.99595891659135 docs/s, 0.12029878842645736 MB/s).
Processed 200 documents (347.26952835539146 docs/s, 0.1846339817601497 MB/s).
Processed 300 documents (454.751231302774 docs/s, 0.2391264436275112 MB/s).
Processed 400 documents (533.7760358421046 docs/s, 0.2853254999253926 MB/s).
Processed 500 documents (607.6853733459014 docs/s, 0.3009936978582454 MB/s).
Processed 600 documents (705.3881876503789 docs/s, 0.3187000091937115 MB/s).
Processed 700 documents (804.1627841878621 docs/s, 0.33327252837427807 MB/s).
Processed 800 documents (875.213218642944 docs/s, 0.3465647397477945 MB/s).
Processed 900 documents (938.5666078892957 docs/s, 0.35846737259184336 MB/s).
Processed 1000 documents (1011.4016784578616 docs/s, 0.3730967361872535 MB/s).
Processed 1100 documents (1059.0062488179028 docs/s, 0.39028761456013134 MB/s).
Processed 1200 documents (1109.0824554761934 docs/s, 0.40537630664938495 MB/s).
Processed 1300 documents (1148.256113185646 docs/s, 0.41975172409

Processing file data/custom_dataset/json/wikinews-zhtw.jsonl 1/1


[NeMo W 2024-12-06 06:59:06 nemo_logging:349] /usr/lib/python3.10/tempfile.py:999: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpmwec6j7e'>
      _warnings.warn(warn_message, ResourceWarning)
    


### 1.3 Download pre-trained model for continued pre-training

In [ ]:
%%bash
export HF_TOKEN=???
HF_MODEL=meta-llama/Llama-3.1-8B-Instruct ## Download from HF
#HF_MODEL=/workspace/Llama-3.1-8B-Instruct/ ## Load from local

huggingface-cli download $HF_MODEL

python /opt/NeMo/scripts/checkpoint_converters/convert_llama_hf_to_nemo.py \
--input_name_or_path $HF_MODEL \
--output_path Llama-3.1-8B-Instruct.nemo \
--llama31 True \
--precision bf16

### 1.4 Run pre-training <a name='s1.4'></a>

In [3]:
%%bash

MODEL_NAME=Llama-3.1-8B
MODEL=Llama-3.1-8B-Instruct.nemo
NUM_GPUS=4
MAX_STEPS=100
MBS=1
GBS=8
TP=4
PP=1
CP=1
LR=1e-4
DATA_SPLITS=\'9990,8,2\'
DATA_PREFIX=[1.0,data/custom_dataset/preprocessed/wikinews_text_document]
export CUDA_DEVICE_MAX_CONNECTIONS=1

python /opt/NeMo/examples/nlp/language_modeling/megatron_gpt_pretraining.py \
--config-path=/opt/NeMo-Framework-Launcher/launcher_scripts/conf/training/llama --config-name=llama3_1_8b \
+base_results_dir=results \
trainer.num_nodes=1 \
trainer.devices=$NUM_GPUS \
trainer.max_steps=$MAX_STEPS \
trainer.limit_val_batches=1 \
trainer.val_check_interval=50 \
exp_manager.explicit_log_dir=/workspace/results/$MODEL_NAME/pretrain \
exp_manager.wandb_logger_kwargs.name=$MODEL_NAME \
exp_manager.resume_if_exists=True \
exp_manager.checkpoint_callback_params.save_nemo_on_train_end=True \
exp_manager.checkpoint_callback_params.model_parallel_size=$(($TP*$PP)) \
model.micro_batch_size=$MBS \
model.global_batch_size=$GBS \
model.tensor_model_parallel_size=$TP \
model.pipeline_model_parallel_size=$PP \
model.context_parallel_size=$CP \
model.init_method_std=0.02 \
model.optim.lr=$LR \
model.data.splits_string=${DATA_SPLITS} \
model.data.data_prefix=${DATA_PREFIX} \
model.data.num_workers=0 \
+model.restore_from_path=$MODEL \
+model.rotary_base=500000.0 \
+model.seq_len_interpolation_factor=8

[NeMo W 2024-12-06 07:43:13 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:280: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, allreduce_dgrad):
    
[NeMo W 2024-12-06 07:43:13 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:290: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-06 07:43:13 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:381: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-06 07:43:13 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:420: FutureWarning: `t

[NeMo I 2024-12-06 07:43:16 megatron_gpt_pretraining:37] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-12-06 07:43:16 megatron_gpt_pretraining:38] 
    run:
      name: llama3_8b
      results_dir: ${base_results_dir}/${.name}
      time_limit: 0-01:30:00
      dependency: singleton
    trainer:
      num_nodes: 1
      devices: 4
      accelerator: gpu
      precision: bf16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: null
      max_steps: 100
      max_time: 05:23:30:00
      log_every_n_steps: 10
      val_check_interval: 50
      limit_val_batches: 1
      limit_test_batches: 50
      accumulate_grad_batches: 1
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: /workspace/results/Llama-3.1-8B/pretrain
      exp_dir: null
      name: megatron_llama
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: nemo_llama_pretrain
        name: Llama-3.1-8B
  

[NeMo W 2024-12-06 07:43:16 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
[NeMo W 2024-12-06 07:43:16 nemo_logging:349] /opt/NeMo/nemo/collections/nlp/parts/nlp_overrides.py:1395: DeprecationWarning: torch.set_autocast_gpu_dtype(dtype) is deprecated. Please use torch.set_autocast_dtype('cuda', dtype) instead. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/autograd/init.cpp:678.)
      torch.set_autocast_gpu_dtype(dtype)
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


[NeMo I 2024-12-06 07:43:17 exp_manager:400] ExpManager schema
[NeMo I 2024-12-06 07:43:17 exp_manager:401] {'explicit_log_dir': None, 'exp_dir': None, 'name': None, 'version': None, 'use_datetime_version': True, 'resume_if_exists': False, 'resume_past_end': False, 'resume_ignore_no_checkpoint': False, 'resume_from_checkpoint': None, 'create_tensorboard_logger': True, 'summary_writer_kwargs': None, 'create_wandb_logger': False, 'wandb_logger_kwargs': None, 'create_mlflow_logger': False, 'mlflow_logger_kwargs': {'experiment_name': None, 'tracking_uri': None, 'tags': None, 'save_dir': './mlruns', 'prefix': '', 'artifact_location': None, 'run_id': None, 'log_model': False}, 'create_dllogger_logger': False, 'dllogger_logger_kwargs': {'verbose': False, 'stdout': False, 'json_file': './dllogger.json'}, 'create_clearml_logger': False, 'clearml_logger_kwargs': {'project': None, 'task': None, 'connect_pytorch': False, 'model_name': None, 'tags': None, 'log_model': False, 'log_cfg': False, 'log_

[NeMo W 2024-12-06 07:43:17 exp_manager:1139] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to 100. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


...

[NeMo W 2024-12-06 07:52:45 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:280: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, allreduce_dgrad):
    
[NeMo W 2024-12-06 07:52:45 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:290: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-06 07:52:45 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:381: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-06 07:52:45 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:420: FutureWarning: `t

Epoch 0: : 100%|██████████| 100/100 [11:39<00:00, reduced_train_loss=2.080, global_step=99.00, consumed_samples=800.0, train_step_timing in s=4.920, val_loss=1.790]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 8/8 [00:01<00:00,  4.65it/s]


Epoch 0, global step 100: 'val_loss' reached 1.65426 (best 1.65426), saving model to '/workspace/results/Llama-3.1-8B/pretrain/checkpoints/megatron_llama--val_loss=1.65-step=100-consumed_samples=800.0.ckpt' as top 10
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid de

Epoch 0: : 100%|██████████| 100/100 [15:17<00:00, reduced_train_loss=2.080, global_step=99.00, consumed_samples=800.0, train_step_timing in s=4.920, val_loss=1.650]
[NeMo I 2024-12-06 08:03:33 perf_metrics_utils:42] train_step_timing in s: [4.87, 4.91, 4.91, 4.92, 4.92, 4.88, 4.91, 4.91, 4.91, 4.92]
[NeMo I 2024-12-06 08:03:33 perf_metrics:86] TFLOPs per sec per GPU=193.40
[NeMo I 2024-12-06 08:03:33 dist_ckpt_io:421] Using TorchDistSaveShardedStrategy(torch_dist, 1) dist-ckpt save strategy.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

## 2. Instruction Tuning <a name='s2'></a>

We will be using the [erhwenkuo/alpaca-data-gpt4-chinese-zhtw](https://huggingface.co/datasets/erhwenkuo/alpaca-data-gpt4-chinese-zhtw) is a dataset that contains Chinese (zh-tw) Instruction-Following generated by GPT-4 using Alpaca prompts for fine-tuning LLMs.

The dataset was originaly shared in [this repository](https://github.com/Instruction-Tuning-with-GPT-4/GPT-4-LLM). This dataset is a translation from English to Chinese.

### 2.1 Download dataset: erhwenkuo/alpaca-data-gpt4-chinese-zhtw <a name='s2.1'></a>
Let's download dataset and save it as json first.

In [4]:
import os
import json
from datasets import load_dataset
dataset = load_dataset('erhwenkuo/alpaca-data-gpt4-chinese-zhtw')['train']
output_path = 'data/alpaca/gpt4-chinese-zhtw.jsonl'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'w') as f:
    for human_instruction, human_input, assistant_output in zip(dataset['instruction'], dataset['input'], dataset['output']):
        f.write(json.dumps({'input': '\n'.join([human_instruction.strip(),human_input.strip()]).strip(), 'output': assistant_output.strip()}, ensure_ascii=False)+ '\n')

Generating train split: 100%|██████████| 52049/52049 [00:00<00:00, 426102.45 examples/s]


In [5]:
!head -n 1 data/alpaca/gpt4-chinese-zhtw.jsonl

{"input": "給出三個保持健康的小貼士。", "output": "1. 飲食要均衡且富有營養：確保你的餐食包含各種水果、蔬菜、瘦肉、全穀物和健康脂肪。這有助於為身體提供必要的營養，使其發揮最佳功能，並有助於預防慢性疾病。2. 經常參加體育鍛煉：鍛鍊對於保持強壯的骨骼、肌肉和心血管健康至關重要。每週至少要進行150分鐘的中等有氧運動或75分鐘的劇烈運動。3. 獲得足夠的睡眠：獲得足夠的高質量睡眠對身體和心理健康至關重要。它有助於調節情緒，提高認知功能，並支援健康的生長和免疫功能。每晚睡眠目標為7-9小時。"}


### 2.2 Split the data into train, validation and test. <a name='s2.2'></a>

Generate the train, test and validation splits- you may use your own script to do this or create a new script and use the following sample split_train_val.py by copying it over in the alpaca directory

In [6]:
import json
import random

input_file = "data/alpaca/gpt4-chinese-zhtw.jsonl"
training_output_file = "data/alpaca/training.jsonl"
validation_output_file = "data/alpaca/validation.jsonl"
test_output_file = "data/alpaca/test.jsonl"

# Specify the proportion of data for training and validation
train_proportion = 0.98
validation_proportion = 0.01
test_proportion = 0.01

# Read the JSONL file and shuffle the JSON objects
with open(input_file, "r") as f:
    lines = f.readlines()
    random.shuffle(lines)

# Calculate split indices
total_lines = len(lines)
train_index = int(total_lines * train_proportion)
val_index = int(total_lines * validation_proportion)

# Distribute JSON objects into training and validation sets
train_data = lines[:train_index]
validation_data = lines[train_index:train_index+val_index]
test_data = lines[train_index+val_index:]

# Write JSON objects to training file
with open(training_output_file, "w") as f:
    for line in train_data:
        f.write(line.strip() + "\n")

# Write JSON objects to validation file
with open(validation_output_file, "w") as f:
    for line in validation_data:
        f.write(line.strip() + "\n")

# Write JSON objects to training file
with open(test_output_file, "w") as f:
    for line in test_data:
        f.write(line.strip() + "\n")

In [7]:
# What the dataset looks like after spliting
!head -1 data/alpaca/training.jsonl

{"input": "從以下列表中，列出三個與健身相關的主題。\n瑜伽，跑步，舉重", "output": "從列表中選擇三個與健身相關的話題：瑜伽、跑步和舉重。"}


### 2.3 Full parameter fine-tuning  <a name='s2.3'></a>

In [9]:
%%bash

MODEL_NAME=Llama-3.1-8B
MODEL=results/Llama-3.1-8B/pretrain/checkpoints/megatron_llama.nemo
NUM_GPUS=4
MAX_STEPS=100
VAL_INTERVAL=1.0
GBS=16
MBS=1
TP=4
PP=1
LR=1e-4
SEQ_LEN=8192
TRAIN_DS=[data/alpaca/training.jsonl]
VALID_DS=[data/alpaca/validation.jsonl]
TEST_DS=[data/alpaca/test.jsonl]
CONCAT_SAMPLING_PROBS=[1.0]
PROMPT_TEMPLATE="\"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\
You are a knowledgeable assistant trained to provide accurate and helpful information. Please respond to the user's queries promptly and politely.<|eot_id|>\n\
<|start_header_id|>user<|end_header_id|>\n\
{input}<|eot_id|>\n\
<|start_header_id|>assistant<|end_header_id|>\n\
{output}\""

python /opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_finetuning.py \
--config-path=/opt/NeMo/examples/nlp/language_modeling/tuning/conf --config-name=megatron_gpt_finetuning_config \
trainer.devices=$NUM_GPUS \
trainer.max_epochs=null \
trainer.max_steps=$MAX_STEPS \
trainer.val_check_interval=$VAL_INTERVAL \
exp_manager.explicit_log_dir=/workspace/results/$MODEL_NAME/SFT \
exp_manager.checkpoint_callback_params.save_nemo_on_train_end=True \
model.tensor_model_parallel_size=$TP \
model.pipeline_model_parallel_size=$PP \
model.restore_from_path=$MODEL \
model.global_batch_size=$GBS \
model.micro_batch_size=$MBS \
model.data.train_ds.file_names=${TRAIN_DS} \
model.data.validation_ds.file_names=${VALID_DS} \
model.data.test_ds.file_names=${TEST_DS} \
model.data.train_ds.max_seq_length=$SEQ_LEN \
model.data.validation_ds.max_seq_length=$SEQ_LEN \
model.data.test_ds.max_seq_length=$SEQ_LEN \
model.data.train_ds.num_workers=0 \
model.data.validation_ds.num_workers=0 \
model.data.test_ds.num_workers=0 \
model.data.train_ds.concat_sampling_probabilities=${CONCAT_SAMPLING_PROBS} \
model.data.train_ds.prompt_template="$PROMPT_TEMPLATE" \
model.optim.lr=$LR \
model.peft.peft_scheme=null

[NeMo W 2024-12-06 08:21:05 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:280: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, allreduce_dgrad):
    
[NeMo W 2024-12-06 08:21:05 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:290: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-06 08:21:05 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:381: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-06 08:21:05 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:420: FutureWarning: `t

[NeMo I 2024-12-06 08:21:09 megatron_gpt_finetuning:56] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-12-06 08:21:09 megatron_gpt_finetuning:57] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 4
      accelerator: gpu
      num_nodes: 1
      precision: 16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: null
      max_steps: 100
      log_every_n_steps: 10
      val_check_interval: 1.0
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: /workspace/results/Llama-3.1-8B/SFT
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${model.data.validation_ds.metric.name}
        save_top_k: 1


[NeMo W 2024-12-06 08:21:09 nemo_logging:349] /opt/NeMo/nemo/collections/nlp/parts/nlp_overrides.py:1451: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
      super().__init__(
    
[NeMo W 2024-12-06 08:21:09 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
[NeMo W 2024-12-06 08:21:09 nemo_logging:349] /opt/NeMo/nemo/collections/nlp/parts/nlp_overrides.py:1395: DeprecationWarning: torch.set_autocast_gpu_dtype(dtype) is deprecated. Please use torch.set_autocast_dtype('cuda', dtype) instead. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/autograd/init.cpp:678.)
      torch.set_autocast_gpu_dtype(dtype)
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val

[NeMo I 2024-12-06 08:21:10 exp_manager:400] ExpManager schema
[NeMo I 2024-12-06 08:21:10 exp_manager:401] {'explicit_log_dir': None, 'exp_dir': None, 'name': None, 'version': None, 'use_datetime_version': True, 'resume_if_exists': False, 'resume_past_end': False, 'resume_ignore_no_checkpoint': False, 'resume_from_checkpoint': None, 'create_tensorboard_logger': True, 'summary_writer_kwargs': None, 'create_wandb_logger': False, 'wandb_logger_kwargs': None, 'create_mlflow_logger': False, 'mlflow_logger_kwargs': {'experiment_name': None, 'tracking_uri': None, 'tags': None, 'save_dir': './mlruns', 'prefix': '', 'artifact_location': None, 'run_id': None, 'log_model': False}, 'create_dllogger_logger': False, 'dllogger_logger_kwargs': {'verbose': False, 'stdout': False, 'json_file': './dllogger.json'}, 'create_clearml_logger': False, 'clearml_logger_kwargs': {'project': None, 'task': None, 'connect_pytorch': False, 'model_name': None, 'tags': None, 'log_model': False, 'log_cfg': False, 'log_

[NeMo W 2024-12-06 08:21:10 exp_manager:784] There were no checkpoints found in checkpoint_dir or no checkpoint folder at checkpoint_dir :/workspace/results/Llama-3.1-8B/SFT/checkpoints. Training from scratch.


...

[NeMo W 2024-12-06 08:29:02 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:280: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, allreduce_dgrad):
    
[NeMo W 2024-12-06 08:29:02 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:290: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-06 08:29:02 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:381: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-06 08:29:02 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:420: FutureWarning: `t

Epoch 0: : 100%|██████████| 100/100 [06:38<00:00, reduced_train_loss=2.850, global_step=99.00, consumed_samples=1600.0, train_step_timing in s=2.230, val_loss=2.730]
[NeMo I 2024-12-06 08:30:49 perf_metrics:86] TFLOPs per sec per GPU=-1.00


[NeMo E 2024-12-06 08:30:49 perf_metrics:84] Failed to calculate TFLOPs per sec per GPU.
    FLOPs measurement not supported for finetuning jobs
Restoring states from the checkpoint path at /workspace/results/Llama-3.1-8B/SFT/checkpoints/megatron_gpt_peft_None_tuning--validation_loss=2.732-step=100-consumed_samples=1600.0
[NeMo W 2024-12-06 08:30:49 nemo_logging:349] /opt/megatron-lm/megatron/core/dist_checkpointing/strategies/torch.py:755: FutureWarning: `load_state_dict` is deprecated and will be removed in future versions. Please use `load` instead.
      checkpoint.load_state_dict(
    
[NeMo W 2024-12-06 08:30:49 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/torch/distributed/checkpoint/planner_helpers.py:311: FutureWarning: Please use DTensor instead and we are deprecating ShardedTensor.
      device = getattr(value, "device", None)
    
Restored all states from the checkpoint at /workspace/results/Llama-3.1-8B/SFT/checkpoints/megatron_gpt_peft_None_tuning--validation

[NeMo I 2024-12-06 08:31:01 dist_ckpt_io:421] Using TorchDistSaveShardedStrategy(torch_dist, 1) dist-ckpt save strategy.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

### 2.4. Parameter Efficient Fine-tuning <a name='s2.4'></a>
Fine-tuning language model can be computationally expensive and risk overfitting, especially with small, specialized datasets. Parameter-efficient fine-tuning methods like LoRA offer a solution. These techniques adapt the model to specific tasks by modifying only a subset of parameters, reducing computational costs and mitigating overfitting risks. In essence, LoRA enable a more efficient and targeted adaptation of large language models for specialized tasks.

In [10]:
%%bash

MODEL_NAME=Llama-3.1-8B
MODEL=results/Llama-3.1-8B/pretrain/checkpoints/megatron_llama.nemo
NUM_GPUS=4
MAX_STEPS=100
VAL_INTERVAL=1.0
GBS=16
MBS=1
TP=4
PP=1
LR=1e-4
SEQ_LEN=8192
TRAIN_DS=[data/alpaca/training.jsonl]
VALID_DS=[data/alpaca/validation.jsonl]
TEST_DS=[data/alpaca/test.jsonl]
CONCAT_SAMPLING_PROBS=[1.0]
PROMPT_TEMPLATE="\"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\
You are a knowledgeable assistant trained to provide accurate and helpful information. Please respond to the user's queries promptly and politely.<|eot_id|>\n\
<|start_header_id|>user<|end_header_id|>\n\
{input}<|eot_id|>\n\
<|start_header_id|>assistant<|end_header_id|>\n\
{output}\""

python /opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_finetuning.py \
--config-path=/opt/NeMo/examples/nlp/language_modeling/tuning/conf --config-name=megatron_gpt_finetuning_config \
trainer.devices=$NUM_GPUS \
trainer.max_epochs=null \
trainer.max_steps=$MAX_STEPS \
trainer.val_check_interval=$VAL_INTERVAL \
exp_manager.explicit_log_dir=/workspace/results/$MODEL_NAME/PEFT \
exp_manager.checkpoint_callback_params.save_nemo_on_train_end=True \
model.tensor_model_parallel_size=1 \
model.restore_from_path=$MODEL \
model.global_batch_size=$GBS \
model.micro_batch_size=$MBS \
model.data.train_ds.file_names=${TRAIN_DS} \
model.data.validation_ds.file_names=${VALID_DS} \
model.data.test_ds.file_names=${TEST_DS} \
model.data.train_ds.max_seq_length=$SEQ_LEN \
model.data.validation_ds.max_seq_length=$SEQ_LEN \
model.data.test_ds.max_seq_length=$SEQ_LEN \
model.data.train_ds.num_workers=0 \
model.data.validation_ds.num_workers=0 \
model.data.test_ds.num_workers=0 \
model.data.train_ds.concat_sampling_probabilities=${CONCAT_SAMPLING_PROBS} \
model.data.train_ds.prompt_template="$PROMPT_TEMPLATE" \
model.optim.lr=$LR \
model.peft.peft_scheme=lora \
model.peft.lora_tuning.adapter_dim=32

[NeMo W 2024-12-06 08:35:57 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:280: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, allreduce_dgrad):
    
[NeMo W 2024-12-06 08:35:57 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:290: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-06 08:35:57 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:381: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-06 08:35:57 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:420: FutureWarning: `t

[NeMo I 2024-12-06 08:36:01 megatron_gpt_finetuning:56] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-12-06 08:36:01 megatron_gpt_finetuning:57] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 4
      accelerator: gpu
      num_nodes: 1
      precision: 16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: null
      max_steps: 100
      log_every_n_steps: 10
      val_check_interval: 1.0
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: /workspace/results/Llama-3.1-8B/PEFT
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${model.data.validation_ds.metric.name}
        save_top_k: 1

[NeMo W 2024-12-06 08:36:01 nemo_logging:349] /opt/NeMo/nemo/collections/nlp/parts/nlp_overrides.py:1451: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
      super().__init__(
    
[NeMo W 2024-12-06 08:36:01 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
[NeMo W 2024-12-06 08:36:01 nemo_logging:349] /opt/NeMo/nemo/collections/nlp/parts/nlp_overrides.py:1395: DeprecationWarning: torch.set_autocast_gpu_dtype(dtype) is deprecated. Please use torch.set_autocast_dtype('cuda', dtype) instead. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/autograd/init.cpp:678.)
      torch.set_autocast_gpu_dtype(dtype)
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val

[NeMo I 2024-12-06 08:36:02 exp_manager:400] ExpManager schema
[NeMo I 2024-12-06 08:36:02 exp_manager:401] {'explicit_log_dir': None, 'exp_dir': None, 'name': None, 'version': None, 'use_datetime_version': True, 'resume_if_exists': False, 'resume_past_end': False, 'resume_ignore_no_checkpoint': False, 'resume_from_checkpoint': None, 'create_tensorboard_logger': True, 'summary_writer_kwargs': None, 'create_wandb_logger': False, 'wandb_logger_kwargs': None, 'create_mlflow_logger': False, 'mlflow_logger_kwargs': {'experiment_name': None, 'tracking_uri': None, 'tags': None, 'save_dir': './mlruns', 'prefix': '', 'artifact_location': None, 'run_id': None, 'log_model': False}, 'create_dllogger_logger': False, 'dllogger_logger_kwargs': {'verbose': False, 'stdout': False, 'json_file': './dllogger.json'}, 'create_clearml_logger': False, 'clearml_logger_kwargs': {'project': None, 'task': None, 'connect_pytorch': False, 'model_name': None, 'tags': None, 'log_model': False, 'log_cfg': False, 'log_

[NeMo W 2024-12-06 08:36:02 exp_manager:784] There were no checkpoints found in checkpoint_dir or no checkpoint folder at checkpoint_dir :/workspace/results/Llama-3.1-8B/PEFT/checkpoints. Training from scratch.


...

[NeMo W 2024-12-06 08:39:15 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-12-06 08:39:15 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('validation_loss_dataloader0', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-12-06 08:39:15 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('validation_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 202

Epoch 0: : 100%|██████████| 100/100 [01:14<00:00, reduced_train_loss=1.400, global_step=99.00, consumed_samples=1600.0, train_step_timing in s=0.557]
Validation: |          | 0/? [00:00<?, ?it/s][NeMo I 2024-12-06 08:40:32 num_microbatches_calculator:218] setting number of microbatches to constant 4

Validation DataLoader 0: 100%|██████████| 33/33 [00:11<00:00,  2.91it/s][NeMo I 2024-12-06 08:40:43 num_microbatches_calculator:218] setting number of microbatches to constant 4



Epoch 0, global step 100: 'validation_loss' reached 1.55920 (best 1.55920), saving model to '/workspace/results/Llama-3.1-8B/PEFT/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=1.559-step=100-consumed_samples=1600.0.ckpt' as top 1
[rank: 2] Metric val_loss improved. New best score: 1.559
[rank: 3] Metric val_loss improved. New best score: 1.559
[rank: 0] Metric val_loss improved. New best score: 1.559
[rank: 1] Metric val_loss improved. New best score: 1.559
`Trainer.fit` stopped: `max_steps=100` reached.


Epoch 0: : 100%|██████████| 100/100 [01:32<00:00, reduced_train_loss=1.400, global_step=99.00, consumed_samples=1600.0, train_step_timing in s=0.557, val_loss=1.560]
[NeMo I 2024-12-06 08:40:49 perf_metrics:86] TFLOPs per sec per GPU=-1.00


[NeMo E 2024-12-06 08:40:49 perf_metrics:84] Failed to calculate TFLOPs per sec per GPU.
    FLOPs measurement not supported for finetuning jobs
Restoring states from the checkpoint path at /workspace/results/Llama-3.1-8B/PEFT/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=1.559-step=100-consumed_samples=1600.0.ckpt
Restored all states from the checkpoint at /workspace/results/Llama-3.1-8B/PEFT/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=1.559-step=100-consumed_samples=1600.0.ckpt
[NeMo W 2024-12-06 08:41:04 nemo_logging:349] /usr/lib/python3.10/tempfile.py:999: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpky8sq6l4'>
      _warnings.warn(warn_message, ResourceWarning)
    


## 3 Evaluation <a name='s3'></a>

If you want to evaluate an SFT .nemo file:

In [12]:
%%bash

MODEL_NAME=Llama-3.1-8B
MODEL=results/Llama-3.1-8B/SFT/checkpoints/megatron_gpt_peft_None_tuning.nemo
NUM_GPUS=4
TP=1
GB=32
SEQ_LEN=8192
TEST_DS=[data/alpaca/test.jsonl]
OUTPUT=data/alpaca/prediction
PROMPT_TEMPLATE="\"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\
You are a knowledgeable assistant trained to provide accurate and helpful information. Please respond to the user's queries promptly and politely.<|eot_id|>\n\
<|start_header_id|>user<|end_header_id|>\n\
{input}<|eot_id|>\n\
<|start_header_id|>assistant<|end_header_id|>\n\
{output}\""

python /opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_generate.py \
trainer.precision=bf16 \
trainer.devices=$NUM_GPUS \
model.restore_from_path=$MODEL \
model.global_batch_size=$GB \
model.tensor_model_parallel_size=$TP \
model.pipeline_model_parallel_size=1 \
model.megatron_amp_O2=True \
model.peft.restore_from_path=null \
model.data.test_ds.file_names=$TEST_DS \
model.data.test_ds.names=\['alpaca_test'] \
model.data.test_ds.global_batch_size=$GB \
model.data.test_ds.tokens_to_generate=128 \
model.data.test_ds.label_key='output' \
model.data.test_ds.add_eos=True \
model.data.test_ds.add_sep=False \
model.data.test_ds.add_bos=False \
model.data.test_ds.max_seq_length=$SEQ_LEN \
model.data.test_ds.truncation_field="input" \
model.data.test_ds.prompt_template="$PROMPT_TEMPLATE" \
model.data.test_ds.write_predictions_to_file=True \
model.data.test_ds.output_file_path_prefix=$OUTPUT

[NeMo W 2024-12-06 08:43:10 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:280: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, allreduce_dgrad):
    
[NeMo W 2024-12-06 08:43:10 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:290: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-06 08:43:10 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:381: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-06 08:43:10 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:420: FutureWarning: `t

[NeMo I 2024-12-06 08:43:13 megatron_gpt_generate:125] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-12-06 08:43:13 megatron_gpt_generate:126] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 4
      accelerator: gpu
      num_nodes: 1
      precision: bf16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 20000
      log_every_n_steps: 10
      val_check_interval: 200
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: null
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${model.data.test_ds.metric.name}
        save_top_k: 1
        mode: max
        save_nemo

[NeMo W 2024-12-06 08:43:13 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
[NeMo W 2024-12-06 08:43:13 nemo_logging:349] /opt/NeMo/nemo/collections/nlp/parts/nlp_overrides.py:1659: DeprecationWarning: torch.set_autocast_gpu_dtype(dtype) is deprecated. Please use torch.set_autocast_dtype('cuda', dtype) instead. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/autograd/init.cpp:678.)
      torch.set_autocast_gpu_dtype(dtype)
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo W 2024-12-06 08:44:08 megatron_base_model:1186] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-06 08:44:08 megatron_base_model:118

[NeMo I 2024-12-06 08:44:08 megatron_init:314] Rank 0 has data parallel group : [0, 1, 2, 3]
[NeMo I 2024-12-06 08:44:08 megatron_init:320] Rank 0 has combined group of data parallel and context parallel : [0, 1, 2, 3]
[NeMo I 2024-12-06 08:44:08 megatron_init:325] All data parallel group ranks with context parallel combined: [[0, 1, 2, 3]]
[NeMo I 2024-12-06 08:44:08 megatron_init:328] Ranks 0 has data parallel rank: 0
[NeMo I 2024-12-06 08:44:08 megatron_init:336] Rank 0 has context parallel group: [0]
[NeMo I 2024-12-06 08:44:08 megatron_init:339] All context parallel group ranks: [[0], [1], [2], [3]]
[NeMo I 2024-12-06 08:44:08 megatron_init:340] Ranks 0 has context parallel rank: 0
[NeMo I 2024-12-06 08:44:08 megatron_init:347] Rank 0 has model parallel group: [0]
[NeMo I 2024-12-06 08:44:08 megatron_init:348] All model parallel group ranks: [[0], [1], [2], [3]]
[NeMo I 2024-12-06 08:44:08 megatron_init:357] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-12-06 08:44:08 m

[NeMo W 2024-12-06 08:44:08 megatron_base_model:1186] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-06 08:44:08 megatron_base_model:1186] The model: MegatronGPTSFTModel() does not have field.name: moe_extended_tp in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-06 08:44:08 megatron_base_model:1186] The model: MegatronGPTSFTModel() does not have field.name: finalize_model_grads_func in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-06 08:44:08 megatron_base_model:1186] The model: MegatronGPTSFTModel() does not have field.name: use_te_rng_tracker in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-06 08:44:08 megatron_base_model:1186] The model: MegatronGPTSFTModel() does not have field.name: tp_comm_bulk_wg

...

[NeMo W 2024-12-06 08:52:55 megatron_gpt_sft_model:677] No training data found, reconfiguring microbatches based on validation batch sizes.


setting number of microbatches to constant 8


[NeMo W 2024-12-06 08:52:55 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-12-06 08:52:55 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_loss_alpaca_test', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-12-06 08:52:55 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    


Testing DataLoader 0: 100%|██████████| 17/17 [04:08<00:00,  0.07it/s]
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    2.7225630283355713     │
│   test_loss_alpaca_test   │    2.7225630283355713     │
│         val_loss          │    2.7225630283355713     │
└───────────────────────────┴───────────────────────────┘


[NeMo W 2024-12-06 08:53:03 nemo_logging:349] /usr/lib/python3.10/tempfile.py:999: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmpdx1n55ja'>
      _warnings.warn(warn_message, ResourceWarning)
    


In [13]:
import json

def modify_and_overwrite_jsonl(file_path):
    data_list = []
    with open(file_path, 'r') as file:
        for line in file:
            data = json.loads(line)
            data_list.append(data)
    
    with open(file_path, 'w', encoding='utf-8') as file:
        for data in data_list:
            json_line = json.dumps(data, ensure_ascii=False) + "\n"
            file.write(json_line)

file_path = "data/alpaca/prediction_test_alpaca_test_inputs_preds_labels.jsonl"
modify_and_overwrite_jsonl(file_path)

If you want to evaluate a PEFT Model, you should provide a base GPT model and a PEFT model .nemo file

In [14]:
%%bash

MODEL_NAME=Llama-3.1-8B
MODEL=results/Llama-3.1-8B/pretrain/checkpoints/megatron_llama.nemo
PEFT_MODEL=results/Llama-3.1-8B/PEFT/checkpoints/megatron_gpt_peft_lora_tuning.nemo
NUM_GPUS=4
GB=32
SEQ_LEN=8192
TEST_DS=[data/alpaca/test.jsonl]
OUTPUT=data/alpaca/prediction_peft
PROMPT_TEMPLATE="\"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\
You are a knowledgeable assistant trained to provide accurate and helpful information. Please respond to the user's queries promptly and politely.<|eot_id|>\n\
<|start_header_id|>user<|end_header_id|>\n\
{input}<|eot_id|>\n\
<|start_header_id|>assistant<|end_header_id|>\n\
{output}\""

python /opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_generate.py \
model.restore_from_path=$MODEL \
model.peft.restore_from_path=$PEFT_MODEL \
model.peft.peft_scheme=lora \
trainer.devices=$NUM_GPUS \
model.global_batch_size=$GB \
model.data.test_ds.file_names=$TEST_DS \
model.data.test_ds.names=\['alpaca_test'] \
model.data.test_ds.global_batch_size=$GB \
model.data.test_ds.tokens_to_generate=128 \
model.data.test_ds.label_key='output' \
model.data.test_ds.add_eos=True \
model.data.test_ds.add_sep=False \
model.data.test_ds.add_bos=False \
model.data.test_ds.max_seq_length=$SEQ_LEN \
model.data.test_ds.truncation_field="input" \
model.data.test_ds.prompt_template="$PROMPT_TEMPLATE" \
model.data.test_ds.write_predictions_to_file=True \
model.data.test_ds.output_file_path_prefix=$OUTPUT

[NeMo W 2024-12-06 09:00:36 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:280: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, allreduce_dgrad):
    
[NeMo W 2024-12-06 09:00:36 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:290: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-12-06 09:00:36 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:381: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-12-06 09:00:36 nemo_logging:349] /opt/megatron-lm/megatron/core/tensor_parallel/layers.py:420: FutureWarning: `t

[NeMo I 2024-12-06 09:00:40 megatron_gpt_generate:125] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-12-06 09:00:40 megatron_gpt_generate:126] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 4
      accelerator: gpu
      num_nodes: 1
      precision: 16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 20000
      log_every_n_steps: 10
      val_check_interval: 200
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: null
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${model.data.test_ds.metric.name}
        save_top_k: 1
        mode: max
        save_nemo_o

[NeMo W 2024-12-06 09:00:40 nemo_logging:349] /opt/NeMo/nemo/collections/nlp/parts/nlp_overrides.py:1451: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
      super().__init__(
    
[NeMo W 2024-12-06 09:00:40 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
[NeMo W 2024-12-06 09:00:40 nemo_logging:349] /opt/NeMo/nemo/collections/nlp/parts/nlp_overrides.py:1395: DeprecationWarning: torch.set_autocast_gpu_dtype(dtype) is deprecated. Please use torch.set_autocast_dtype('cuda', dtype) instead. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/autograd/init.cpp:678.)
      torch.set_autocast_gpu_dtype(dtype)
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo W 2024

[NeMo I 2024-12-06 09:01:07 megatron_init:314] Rank 0 has data parallel group : [0, 1, 2, 3]
[NeMo I 2024-12-06 09:01:07 megatron_init:320] Rank 0 has combined group of data parallel and context parallel : [0, 1, 2, 3]
[NeMo I 2024-12-06 09:01:07 megatron_init:325] All data parallel group ranks with context parallel combined: [[0, 1, 2, 3]]
[NeMo I 2024-12-06 09:01:07 megatron_init:328] Ranks 0 has data parallel rank: 0
[NeMo I 2024-12-06 09:01:07 megatron_init:336] Rank 0 has context parallel group: [0]
[NeMo I 2024-12-06 09:01:07 megatron_init:339] All context parallel group ranks: [[0], [1], [2], [3]]
[NeMo I 2024-12-06 09:01:07 megatron_init:340] Ranks 0 has context parallel rank: 0
[NeMo I 2024-12-06 09:01:07 megatron_init:347] Rank 0 has model parallel group: [0]
[NeMo I 2024-12-06 09:01:07 megatron_init:348] All model parallel group ranks: [[0], [1], [2], [3]]
[NeMo I 2024-12-06 09:01:07 megatron_init:357] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-12-06 09:01:07 m

[NeMo W 2024-12-06 09:01:07 megatron_base_model:1186] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-06 09:01:07 megatron_base_model:1186] The model: MegatronGPTSFTModel() does not have field.name: moe_extended_tp in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-06 09:01:07 megatron_base_model:1186] The model: MegatronGPTSFTModel() does not have field.name: finalize_model_grads_func in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-06 09:01:07 megatron_base_model:1186] The model: MegatronGPTSFTModel() does not have field.name: use_te_rng_tracker in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-06 09:01:07 megatron_base_model:1186] The model: MegatronGPTSFTModel() does not have field.name: tp_comm_bulk_wg

...

[NeMo W 2024-12-06 09:08:22 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/transformer_engine/pytorch/module/base.py:578: DeprecationWarning: torch.get_autocast_gpu_dtype() is deprecated. Please use torch.get_autocast_dtype('cuda') instead. (Triggered internally at /opt/pytorch/pytorch/torch/csrc/autograd/init.cpp:733.)
      self.activation_dtype = torch.get_autocast_gpu_dtype()
    


Testing DataLoader 0:  88%|████████▊ | 15/17 [04:43<00:37,  0.05it/s]setting number of microbatches to constant 1


[NeMo W 2024-12-06 09:08:25 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/transformer_engine/pytorch/attention.py:4155: UserWarning: window_size should be (-1, -1) or (>=0, >=0) for attn_mask_type=no_mask
      warnings.warn(
    


setting number of microbatches to constant 8
Testing DataLoader 0:  94%|█████████▍| 16/17 [05:02<00:18,  0.05it/s]setting number of microbatches to constant 1
setting number of microbatches to constant 8
Testing DataLoader 0: 100%|██████████| 17/17 [05:16<00:00,  0.05it/s][NeMo I 2024-12-06 09:09:01 megatron_gpt_sft_model:553] skipping autogenerated example example <|start_header_id|>system<|end_header_id|>
    You are a knowledgeable assistant trained to provide accurate and helpful information. Please respond to the user's queries promptly and politely.<|eot_id|>
    <|start_header_id|>user<|end_header_id|>
    給定一份財務資料樣本，計算每月總支出。
    住房：$1,500，食品：$600，交通：$250，娛樂：$400<|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
     prediction 每月總支出為$2,750。 label 在這種情況下，每個類別的值之和將是每月的總支出：住房+食品+交通+娛樂= $1,500 + $600 + $250 + $400 = $2,750。
[NeMo I 2024-12-06 09:09:01 megatron_gpt_sft_model:553] skipping autogenerated example example <|start_header_id|>system<|end_header_id|>
    You are 

[NeMo W 2024-12-06 09:09:01 megatron_gpt_sft_model:677] No training data found, reconfiguring microbatches based on validation batch sizes.
[NeMo W 2024-12-06 09:09:01 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-12-06 09:09:01 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_loss_alpaca_test', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-12-06 09:09:01 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:431: It is recommended to use `self.log('test_loss', ..

In [15]:
file_path = "data/alpaca/prediction_peft_test_alpaca_test_inputs_preds_labels.jsonl"
modify_and_overwrite_jsonl(file_path)

## 4. Export and Deploy a NeMo Checkpoint to TensorRT-LLM <a name='s4'></a>

Open a terminal and run the following code:

```sh
python /opt/NeMo/scripts/deploy/nlp/deploy_triton.py \
--nemo_checkpoint results/Llama-3.1-8B/SFT/checkpoints/megatron_gpt_peft_None_tuning.nemo \
--model_type llama \
--dtype bfloat16 \
--triton_model_name Llama
```

The command above launches a inference server. Keep it running and run the following cell to send a request to the server.

In [ ]:
%%bash

PROMPT_TEMPLATE="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\
You are a knowledgeable assistant trained to provide accurate and helpful information. Please respond to the user's queries promptly and politely.<|eot_id|>\n\
<|start_header_id|>user<|end_header_id|>\n\
{input}<|eot_id|>\n\
<|start_header_id|>assistant<|end_header_id|>\n"

INPUT="今天天氣好嗎?"

PROMPT="${PROMPT_TEMPLATE//\{input\}/$INPUT}"

python /opt/NeMo/scripts/deploy/nlp/query.py \
--url "http://localhost:8000" \
--model_name Llama \
--prompt "$PROMPT"